In [12]:
import duckdb
import pandas as pd
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [14]:
%sql duckdb:///:default:
# %sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

There's a new jupysql version available (0.10.10), you're running 0.10.2. To upgrade: pip install jupysql --upgrade


In [15]:
# Obtain credentials to access Minio store
from stelar.auth import Realm
from stelar.store import S3Store

In [16]:
stelarstaging2_realm = Realm("https://authst.vsamtuc.top/", "stelarstaging2")
stelarstaging2_realm.well_known()

{'issuer': 'https://authst.vsamtuc.top/realms/stelarstaging2',
 'authorization_endpoint': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/auth',
 'token_endpoint': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/token',
 'introspection_endpoint': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/token/introspect',
 'userinfo_endpoint': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/userinfo',
 'end_session_endpoint': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/logout',
 'frontchannel_logout_session_supported': True,
 'frontchannel_logout_supported': True,
 'jwks_uri': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/certs',
 'check_session_iframe': 'https://authst.vsamtuc.top/realms/stelarstaging2/protocol/openid-connect/login-status-iframe.html',
 'grant_types_supported': ['authorization_code',
  'implicit',
  'refresh_token',
  

In [17]:
miniost_secret = "AmNzFfupuvN4OVs9loWl5F3714LNDGvz"
miniost_client = stelarstaging2_realm.client("miniost", miniost_secret)


In [18]:
alice_token = miniost_client.token("alice", "stelar_alice")
alice_token

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJFSVBtTkZHRjlZV1BBbTdVbmllNE1tNmQtRDRlU0FhcDc5NTl4dmtKTmlFIn0.eyJleHAiOjE3MDk1ODMzMTEsImlhdCI6MTcwOTU3OTcxMSwianRpIjoiOWU3MWZjMTItODc0Mi00M2MyLTlhYjQtOWEwMzc4MmJlNWRjIiwiaXNzIjoiaHR0cHM6Ly9hdXRoc3QudnNhbXR1Yy50b3AvcmVhbG1zL3N0ZWxhcnN0YWdpbmcyIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImRkODMwNDZkLTM4NjctNGUzNy05MDk3LWRkZDVmY2U2MDY0NiIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1pbmlvc3QiLCJzZXNzaW9uX3N0YXRlIjoiZDI4YmY2NTAtOWMxNi00MDEwLTllOTktMTZjZTVmYmU0YzI5IiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLXN0ZWxhcnN0YWdpbmcyIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoicHJvZmlsZSBlbWFpbCBtaW5pby1hdXRob3JpemF0aW9uIiwic2lkIjoiZDI4YmY2NTAtOWMxNi00MDEwLTllOTktMTZjZTVmYmU0YzI5IiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5hbWUiOiJBbGljZSBTdGVsYXJVc2VyIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiYWxpY2UiLC

In [19]:
test_minio_tenant = S3Store("https://miniost.vsamtuc.top")
alice_jwt_token = alice_token['access_token']
alice_role = test_minio_tenant.assume_role_with_web_identity(alice_jwt_token)
alice_role

{'AssumeRoleWithWebIdentityResponse': {'@xmlns': 'https://sts.amazonaws.com/doc/2011-06-15/',
  'AssumeRoleWithWebIdentityResult': {'AssumedRoleUser': {'Arn': None,
    'AssumeRoleId': None},
   'Credentials': {'AccessKeyId': '2IA6PWKUO21CBKJ1YYSL',
    'SecretAccessKey': 'vbzpTDMqGpF8YJEIXKkRaEDLfrwPmULCZ63aNLz3',
    'SessionToken': 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIySUE2UFdLVU8yMUNCS0oxWVlTTCIsImFjciI6IjEiLCJhdWQiOiJhY2NvdW50IiwiYXpwIjoibWluaW9zdCIsImVtYWlsIjoiYWxpY2VAc29mdG5ldC50dWMuZ3IiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzA5NTgzMzExLCJmYW1pbHlfbmFtZSI6IlN0ZWxhclVzZXIiLCJnaXZlbl9uYW1lIjoiQWxpY2UiLCJpYXQiOjE3MDk1Nzk3MTEsImlzcyI6Imh0dHBzOi8vYXV0aHN0LnZzYW10dWMudG9wL3JlYWxtcy9zdGVsYXJzdGFnaW5nMiIsImp0aSI6IjllNzFmYzEyLTg3NDItNDNjMi05YWI0LTlhMDM3ODJiZTVkYyIsIm5hbWUiOiJBbGljZSBTdGVsYXJVc2VyIiwicG9saWN5IjoicmVhZHdyaXRlIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiYWxpY2UiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1zdGVsYXJzdGFnaW5nMiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX

In [20]:
alice_credentials = alice_role['AssumeRoleWithWebIdentityResponse']['AssumeRoleWithWebIdentityResult']['Credentials']
alice_access_key_id = alice_credentials['AccessKeyId']
alice_secret_access_key = alice_credentials['SecretAccessKey']
alice_session_token = alice_credentials['SessionToken']


In [21]:
%%sql
SET s3_region= 'us-east-1';
SET s3_endpoint= 'miniost.vsamtuc.top';
SET s3_url_style= 'path';
SET s3_access_key_id= '{{alice_access_key_id}}';
SET s3_secret_access_key='{{alice_secret_access_key}}';
SET s3_session_token= '{{alice_session_token}}';


,Success


In [22]:
%%sql 
SELECT * 
FROM  's3://bucket001/myslides/psl-state-transitions.parquet'
LIMIT 5

,Year,Technical field,Product type,First filing indicator,Doublure-nu indicator,From phase,From state,To phase,To state,Transition probability,Examiner action,Reward value,Effort,Priority group,Limit date,Delay distribution type,Delay distribution parameters
0,2019,22,E-PCT bis,False,False,1_S,NEW,1_S,awaiting B milestone,1.000000,False,0.0,0.000000,NaN,NaN,discrete,"{""values"": [0.5256673511293635, 8.279260780287..."
1,2019,22,E-PCT bis,False,False,1_S,awaiting B milestone,1_S,awaiting Search,0.984302,False,0.0,0.000000,1.0,5.5,deterministic,"{""delay"": 0.0}"
2,2019,22,E-PCT bis,False,False,1_S,awaiting B milestone,1_S,awaiting E milestone,0.015698,False,0.0,0.000000,NaN,NaN,deterministic,"{""delay"": 6.0}"
3,2019,22,E-PCT bis,False,False,1_S,awaiting Search,1_S,awaiting D milestone,0.989537,True,1.0,1.992415,NaN,NaN,lognorm,"{""s"": 1.2300744277704367, ""loc"": -0.0097760100..."
4,2019,22,E-PCT bis,False,False,1_S,awaiting Search,1_S,awaiting E milestone,0.010463,True,0.0,0.000000,NaN,NaN,deterministic,"{""delay"": 6.0}"
